In [1]:
!pip install phe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 925.9 kB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from scipy.special import expit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from phe import paillier
import time
import psutil
import os
from threading import Thread, Lock

In [3]:
X_train = pd.read_csv('X_train1.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train1.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test1.csv')
X_test=X_test.to_numpy()
y_test = pd.read_csv('y_test1.csv')
y_test=y_test.to_numpy().reshape(-1)

y_train = np.where(y_train == 0, -1, 1)
y_test = np.where(y_test == 0, -1, 1)

n_samples,n_attributes=X_train.shape
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# X_train = np.round(X_train,6)
bias = 0
lr = 0.001
lambda_param=0.01

In [4]:
# Paillier key generation
public_key, private_key = paillier.generate_paillier_keypair()

In [5]:
# Lock for controlling CPU monitoring
monitor_lock = Lock()

In [6]:
# Function to handle encrypted multiplication at the edge device
def paillier_multiplication_edge(a, b):
    with monitor_lock:  # Acquire lock to exclude this part from CPU and RAM monitoring
        a = private_key.decrypt(a)
        a=round(a,5)
        a = a * b
    return a

In [7]:
# Function to handle encrypted multiplication at the cloud
def paillier_multiplication_cloud(a, b):
    noise1 = 0.01
    noise2 = 0.02
    a = a + noise1
    sub_mass = (b * noise1) * (-1)
    sol = paillier_multiplication_edge(a, b)
    sol = sol + sub_mass
    return sol

In [8]:
# enc_X_train = [[public_key.encrypt(j) for j in i] for i in X_train]
# enc_y_train = [public_key.encrypt(int(i)) for i in y_train]
enc_X_train=[]
count=0
for i in X_train:
  row_in=[]
  for j in i:
    row_in.append(public_key.encrypt(j))
  print(f"row {count} encrypted")
  enc_X_train.append(row_in)
  count+=1

enc_y_train = [public_key.encrypt(int(i)) for i in y_train]

row 0 encrypted
row 1 encrypted
row 2 encrypted
row 3 encrypted
row 4 encrypted
row 5 encrypted
row 6 encrypted
row 7 encrypted
row 8 encrypted
row 9 encrypted
row 10 encrypted
row 11 encrypted
row 12 encrypted
row 13 encrypted
row 14 encrypted
row 15 encrypted
row 16 encrypted
row 17 encrypted
row 18 encrypted
row 19 encrypted
row 20 encrypted
row 21 encrypted
row 22 encrypted
row 23 encrypted
row 24 encrypted
row 25 encrypted
row 26 encrypted
row 27 encrypted
row 28 encrypted
row 29 encrypted
row 30 encrypted
row 31 encrypted
row 32 encrypted
row 33 encrypted
row 34 encrypted
row 35 encrypted
row 36 encrypted
row 37 encrypted
row 38 encrypted
row 39 encrypted
row 40 encrypted
row 41 encrypted
row 42 encrypted
row 43 encrypted
row 44 encrypted
row 45 encrypted
row 46 encrypted
row 47 encrypted
row 48 encrypted
row 49 encrypted
row 50 encrypted
row 51 encrypted
row 52 encrypted
row 53 encrypted
row 54 encrypted
row 55 encrypted
row 56 encrypted
row 57 encrypted
row 58 encrypted
row 59 

In [9]:
len(enc_X_train)

360

In [10]:
def paillier_scalar_rescaling(r):
  r=private_key.decrypt(r)
  r=round(r,6)
  r=public_key.encrypt(r)
  return r

In [11]:
# margin calculation:
weights=[]
for i in range(X_train.shape[1]):
  weights.append(public_key.encrypt(0))

weights

In [12]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        # with monitor_lock:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [13]:
pid = os.getpid()

# Duration to monitor CPU usage
monitor_duration = 10500

In [14]:
time.sleep(15)

In [15]:
def dot_prod(a,b):
  s=0
  for i,j in zip(a,b):
    p=paillier_multiplication_cloud(i,j)
    s=s+p
  return s

In [16]:
def marginal_decrypt(p):
  q=None
  r=None
  with monitor_lock:
    q=private_key.decrypt(p)
    if q>=1:
      r=True
    else:
      r=False
  return r


In [17]:
results={}
def fit_svm(enc_X_train,enc_y_train,weights,bias,lr=0.001,lambda_param=0.01,epochs=2):
  train_time_start=time.time()
  l=len(enc_X_train)
  for epoch in range(epochs):
    for u in range(l):

      x_i=enc_X_train[u]
      y_i=enc_y_train[u]

      c1=(dot_prod(x_i,weights)-bias)
      c=paillier_multiplication_cloud(c1,y_i)
      # print(c)
      # print(private_key.decrypt(c))

      if marginal_decrypt(c):
        m1=[]
        for i in weights:
          q=2*lambda_param*lr*i
          m1.append(q)

        m2=[]
        for i,j in zip(weights,m1):
          m2.append(i-j)

        weights=m2

      else:
        #updating weights
        r1=[]
        for i in weights:
          r1.append(2*lambda_param*i)
        # print("r1 elements:")
        # for p in r1:
          # print(private_key.decrypt(p))

        r2=[]
        for i in x_i:
          r2.append(paillier_multiplication_cloud(y_i,i))
        # print("r2 elements:")
        # for p in r2:
          # print(private_key.decrypt(p))

        r3=[]
        for i,j in zip(r1,r2):
          r=i-j
          q=lr*r
          r3.append(q)
        # print("r3 elements:")
        # for p in r3:
          # print(private_key.decrypt(p))

        r4=[]
        for i,j in zip(weights,r3):
          r4.append(i-j)
        # print("r4 elements:")
        # for p in r4:
          # print(private_key.decrypt(p))

        weights=r4
        # print("weights after iteration:")
        # for i in weights:
          # print(private_key.decrypt(i))

        # updating bias
        bias=bias-(lr*y_i)
        # print(f"bias={private_key.decrypt(bias)}")

        if u%5==0:
          weights2=[]
          for i in weights:
            weights2.append(paillier_scalar_rescaling(i))
          weights=weights2
          bias=paillier_scalar_rescaling(bias)

        print(f"epoch={epoch}, iter={u} executed")

  train_time_end=time.time()
  results['train_time']=train_time_end-train_time_start
  results['weights_enc']=weights
  results['bias_enc']=bias
  print(results['train_time'])

In [18]:
train_thread = Thread(target=fit_svm, args=(enc_X_train,enc_y_train,weights,bias,0.001,0.01,2))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")

Streaming output truncated to the last 5000 lines.
CPU utilization: 99.9% | RAM utilization: 1.6739224977916491%
CPU utilization: 98.8% | RAM utilization: 1.6739224977916491%
CPU utilization: 99.6% | RAM utilization: 1.6739224977916491%
epoch=1, iter=152 executed
CPU utilization: 99.6% | RAM utilization: 1.6739224977916491%
CPU utilization: 99.9% | RAM utilization: 1.6739224977916491%
CPU utilization: 106.3% | RAM utilization: 1.6739224977916491%
CPU utilization: 101.7% | RAM utilization: 1.6739224977916491%
CPU utilization: 96.6% | RAM utilization: 1.6739224977916491%
CPU utilization: 103.8% | RAM utilization: 1.6739224977916491%
CPU utilization: 96.4% | RAM utilization: 1.6739224977916491%
CPU utilization: 100.6% | RAM utilization: 1.6739224977916491%
CPU utilization: 107.8% | RAM utilization: 1.6739224977916491%
CPU utilization: 104.5% | RAM utilization: 1.6739224977916491%
CPU utilization: 106.7% | RAM utilization: 1.6739224977916491%
epoch=1, iter=154 executed
CPU utilization: 99.

In [19]:
results_unenc=[]
for i in results['weights_enc']:
  t=private_key.decrypt(i)
  results_unenc.append(t)

In [20]:
results_unenc

[0.06879200692609412,
 0.13065350821333982,
 -0.03854213915891689,
 0.10886804013049502,
 -0.06047743832660201,
 0.014138192502570557,
 -0.013860755660390927,
 0.059163917828471836,
 0.23571724203875902,
 0.13164195844639265,
 0.158974459279105,
 0.15287987090143318,
 0.12739246244856525,
 0.08229550317844372,
 0.1105469323516978,
 0.09489174748473651,
 0.10064877515402024,
 -0.062413312664418234,
 -0.07691665076316948,
 -0.14652893088815802,
 -0.16085569426082524,
 -0.06827718119484719]

In [21]:
enc_X_test=[]
count=0
for i in X_test:
  row_in=[]
  for j in i:
    row_in.append(public_key.encrypt(j))
  print(f"row {count} encrypted")
  enc_X_test.append(row_in)
  count+=1

row 0 encrypted
row 1 encrypted
row 2 encrypted
row 3 encrypted
row 4 encrypted
row 5 encrypted
row 6 encrypted
row 7 encrypted
row 8 encrypted
row 9 encrypted
row 10 encrypted
row 11 encrypted
row 12 encrypted
row 13 encrypted
row 14 encrypted
row 15 encrypted
row 16 encrypted
row 17 encrypted
row 18 encrypted
row 19 encrypted
row 20 encrypted
row 21 encrypted
row 22 encrypted
row 23 encrypted
row 24 encrypted
row 25 encrypted
row 26 encrypted
row 27 encrypted
row 28 encrypted
row 29 encrypted
row 30 encrypted
row 31 encrypted
row 32 encrypted
row 33 encrypted
row 34 encrypted
row 35 encrypted
row 36 encrypted
row 37 encrypted
row 38 encrypted
row 39 encrypted
row 40 encrypted
row 41 encrypted
row 42 encrypted
row 43 encrypted
row 44 encrypted
row 45 encrypted
row 46 encrypted
row 47 encrypted
row 48 encrypted
row 49 encrypted
row 50 encrypted
row 51 encrypted
row 52 encrypted
row 53 encrypted
row 54 encrypted
row 55 encrypted
row 56 encrypted
row 57 encrypted
row 58 encrypted
row 59 

In [22]:
def test_edge_service(appx):
  appx=private_key.decrypt(appx)
  return np.sign(appx)

In [23]:
def predict(enc_X_test,weights,bias):
  unenc_preds=[]
  for i in enc_X_test:
    appx=dot_prod(i,weights)-bias
    appx=test_edge_service(appx)
    unenc_preds.append(appx)
  return unenc_preds

In [24]:
test_time_start=time.time()
y_pred=predict(enc_X_test,results['weights_enc'],results['bias_enc'])
test_time_end=time.time()

results['test_time']=test_time_end-test_time_start
results['test_time']

294.43003487586975

In [25]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.7666666666666667

In [26]:
import joblib
joblib.dump((results_unenc, private_key.decrypt(results['bias_enc']), accuracy, y_pred, cpu_seconds, cpu_usages, ram_usages, results['train_time'], results['test_time']), 'variables_paillier.pkl')

['variables_paillier.pkl']

In [27]:
time.sleep(11600)

KeyboardInterrupt: 